In [8]:
# Dependency versions can be modified here if necessary

# !pip install -I git+https://github.com/cloudera/cmlextensions.git --quiet
# !pip install -I ray[client]==2.6.0 --quiet

In [1]:
import cmlextensions_custom.src.cmlextensions.ray_cluster as rc
import os
import cmlapi
import json
from pprint import pprint
import ray

In [2]:
HOST = os.getenv("CDSW_API_URL").split(":")[0] + "://" + os.getenv("CDSW_DOMAIN")
USERNAME = os.getenv("CDSW_PROJECT_URL").split("/")[6]  # args.username  # "vdibia"
API_KEY = os.getenv("CDSW_APIV2_KEY")
PROJECT_NAME = os.getenv("CDSW_PROJECT")
PROJECT_ID=os.getenv("CDSW_PROJECT_ID")

cml = cmlapi.default_client(url=HOST,cml_api_key=API_KEY)

def set_environ(Cml,Item,Value):
    Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
    if Project.environment=='':
        Project_Environment={}
    else:
        Project_Environment=json.loads(Project.environment)
    Project_Environment[Item]=Value
    Project.environment=json.dumps(Project_Environment)
    Cml.update_project(Project,project_id=os.getenv("CDSW_PROJECT_ID"))

def get_environ(Cml,Item):
    Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
    Project_Environment=json.loads(Project.environment)
    return Project_Environment[Item]

In [3]:
head_system_configs='{"object_spilling_config":"{\"type\":\"filesystem\",\"params\":{\"directory_path\":\"/tmp\"}}"}'

In [4]:
cluster = rc.RayCluster( num_workers=2,
                         worker_cpu=1, worker_memory=1, worker_nvidia_gpu=0, 
                         head_cpu=2, head_memory=4, head_nvidia_gpu=0
                       )
cluster.init()
#set_environ(cml,"RAY_ADDRESS",cluster.get_client_url())

Starting ray head...
Starting 2 ray workers...

--------------------
Ray cluster started
--------------------

The Ray dashboard is running at
https://dlkhvaff2btr6tvn.ml-6e045173-d3d.rags-ita.a465-9q4k.cloudera.site/

To connect to this Ray cluster from this CML Session,
use the following Python code:
  import ray
  ray.init(address='ray://100.100.117.111:10001')



The following initializes a Ray cluster, which can be customized to the size of workload you need to run. To troubleshoot starting up the Ray cluster, refer to the README

In [5]:
ray.init(address=cluster.get_client_url())

I0000 00:00:1722113710.502549    2994 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


Python version:,3.10.13
Ray version:,2.6.0


2024-07-27 20:56:28,823	ERROR dataclient.py:330 -- Unrecoverable error in data channel.


In [6]:
import modin.pandas as pd

In [7]:
pd.read_csv("data/transactions.csv")

,2282592795180240,VISA 16 digit,purchase,2023-03-11T22:35:00.000Z,-102.0,31.3963,MEX,11045
0,60455561336,Discover,purchase,2023-07-05T07:09:00.000Z,-85.0,43.3963,USD,11907
1,4572904481771,Mastercard,purchase,2023-12-25T23:13:00.000Z,-108.0,36.3963,EUR,28466
2,4600480825566739779,JCB 15 digit,purchase,2023-07-10T00:32:00.000Z,-116.0,48.3963,KWD,16777
3,676353427898,Mastercard,purchase,2023-10-26T07:24:00.000Z,-110.0,42.3963,BHD,28483
4,3560436298420474,Discover,purchase,2023-01-31T01:38:00.000Z,-71.0,39.3963,GBP,14215
...,...,...,...,...,...,...,...,...
9994,180053881503461,JCB 16 digit,purchase,2023-11-15T22:19:00.000Z,-122.0,29.3963,CHF,22548
9995,6011459234593871,VISA 19 digit,purchase,2023-04-16T05:37:00.000Z,-88.0,25.3963,MEX,20656
9996,3512935682002383,American Express,purchase,2023-08-26T09:05:00.000Z,-105.0,31.3963,USD,6964
9997,676162369752,VISA 16 digit,purchase,2023-04-24T10:25:00.000Z,-67.0,29.3963,EUR,20415


In [8]:
import modin.pandas as pd
import numpy as np
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # 40x2GB frames -- Working!
df.info()

ConnectionError: Request can't be sent because the Ray client has already been disconnected due to an error. Last exception: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Attempted to reconnect a session that has already been cleaned up"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Attempted to reconnect a session that has already been cleaned up", grpc_status:5, created_time:"2024-07-27T20:56:28.698549225+00:00"}"
>

In [ ]:
#ray.shutdown()
#cluster.terminate()